## Librerías y acceso

In [1]:
import pandas as pd
import requests
import json
import datetime
import finnhub
import yfinance as yf
import numpy as np
import random

FH_token = ''
finnhub_client = finnhub.Client(api_key=FH_token)

## Tickers y sectores

Voy a quedarme con un único sector, y tomar 10 activos para comparar rendimientos con el de ese promedio. Buscamos activos que les vaya mejor que otros similares por sus fundamentos como empresa.

In [2]:
import requests
from bs4 import BeautifulSoup
from random import sample

table = BeautifulSoup(requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies').text, 'lxml').find('table', {'class': 'wikitable sortable'})
tickers = []
sectors = []
for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text
        sector = row.findAll('td')[3].text
        ticker = ticker[:-1]
        sectors.append(sector)
        tickers.append(ticker)
#tickers_sp500 = tickers
#df = pd.DataFrame({'Ticker': tickers, 'Sector': sectors})
#tickers = df['Ticker']

# Construcción de la base de indicadores fundamentales

Ver qué significa cada cosa en https://www.macrotrends.net/stocks/charts/GOOGL/alphabet/pe-ratio

In [3]:
# Descarga datos de balances
import time

full_data_balances = pd.DataFrame()

for t in tickers:
    try:
        print(t)
        js = finnhub_client.company_basic_financials(t, 'all')
        full_data_ticker = pd.DataFrame()
        series = js['series']['quarterly']
        for i in series.keys():
            data_serie = pd.DataFrame(series[i])
            if len(data_serie) == 0:
                pass
            else:
                data_serie.columns = ['fecha_fin_cuatrimestre_fiscal', i]
                if len(full_data_ticker) == 0:
                    full_data_ticker = data_serie.copy()
                else:
                    full_data_ticker = pd.merge(full_data_ticker, data_serie, how='outer', on=['fecha_fin_cuatrimestre_fiscal'])
        full_data_ticker.insert(0,'ticker',t)
        full_data_balances = pd.concat([full_data_balances, full_data_ticker])
    except:
        print('Falló la API. Esperando 60 segundos')
        time.sleep(60)
        print(t)
        js = finnhub_client.company_basic_financials(t, 'all')
        try:
            full_data_ticker = pd.DataFrame()
            series = js['series']['quarterly']
            for i in series.keys():
                data_serie = pd.DataFrame(series[i])
                if len(data_serie) == 0:
                    pass
                else:
                    data_serie.columns = ['fecha_fin_cuatrimestre_fiscal', i]
                    if len(full_data_ticker) == 0:
                        full_data_ticker = data_serie.copy()
                    else:
                        full_data_ticker = pd.merge(full_data_ticker, data_serie, how='outer', on=['fecha_fin_cuatrimestre_fiscal'])
            full_data_ticker.insert(0,'ticker',t)
            full_data_balances = pd.concat([full_data_balances, full_data_ticker])

            continue
        except:
            pass

full_data_balances = full_data_balances.sort_values(by=['ticker', 'fecha_fin_cuatrimestre_fiscal']).reset_index(drop=True)
full_data_balances.insert(0,'fecha_aproximada_presentacion',pd.to_datetime(full_data_balances['fecha_fin_cuatrimestre_fiscal']) + datetime.timedelta(days=45))
full_data_balances['fecha_fin_cuatrimestre_fiscal'] = pd.to_datetime(full_data_balances['fecha_fin_cuatrimestre_fiscal'])

MMM
AOS
ABT
ABBV
ACN
ATVI
ADM
ADBE
ADP
AAP
AES
AFL
A
APD
AKAM
ALK
ALB
ARE
ALGN
ALLE
LNT
ALL
GOOGL
GOOG
MO
AMZN
AMCR
AMD
AEE
AAL
AEP
AXP
AIG
AMT
AWK
AMP
ABC
AME
AMGN
APH
ADI
ANSS
AON
APA
AAPL
AMAT
APTV
ACGL
ANET
AJG
AIZ
T
ATO
ADSK
AZO
AVB
AVY
BKR
BALL
BAC
BBWI
Falló la API. Esperando 60 segundos
BBWI
BAX
BDX
WRB
BRK.B
BBY
BIO
TECH
BIIB
BLK
BK
BA
BKNG
BWA
BXP
BSX
BMY
AVGO
BR
BRO
BF.B
CHRW
CDNS
CZR
CPT
CPB
COF
CAH
KMX
CCL
CARR
CTLT
CAT
CBOE
CBRE
CDW
CE
CNC
CNP
CDAY
CF
CRL
SCHW
CHTR
CVX
CMG
CB
CHD
CI
CINF
CTAS
CSCO
C
CFG
CLX
CME
CMS
KO
CTSH
CL
CMCSA
Falló la API. Esperando 60 segundos
CMCSA
CMA
CAG
COP
ED
STZ
CEG
COO
CPRT
GLW
CTVA
CSGP
COST
CTRA
CCI
CSX
CMI
CVS
DHI
DHR
DRI
DVA
DE
DAL
XRAY
DVN
DXCM
FANG
DLR
DFS
DISH
DIS
DG
DLTR
D
DPZ
DOV
DOW
DTE
DUK
DD
DXC
EMN
ETN
EBAY
ECL
EIX
EW
EA
ELV
LLY
EMR
ENPH
ETR
EOG
EPAM
EQT
EFX
EQIX
EQR
ESS
Falló la API. Esperando 60 segundos
ESS
EL
ETSY
RE
EVRG
ES
EXC
EXPE
EXPD
EXR
XOM
FFIV
FDS
FAST
FRT
FDX
FITB
FRC
FSLR
FE
FIS
FISV
FLT
FMC
F
FTNT
FTV
FOXA
FOX
BEN

In [4]:
# Descarga serie de precios

df_global = pd.DataFrame(columns=['Date', 'ticker', 'Adj Close', 'Target_1Y'])
cuantos_va = 0
for ticker in full_data_balances.ticker.unique():
    print(cuantos_va, len(full_data_balances.ticker.unique()))
    cuantos_va=cuantos_va+1
    df = yf.download(ticker, start=full_data_balances.fecha_fin_cuatrimestre_fiscal.min(), end=full_data_balances.fecha_fin_cuatrimestre_fiscal.max()).reset_index()
    df = df[df.Volume > 0].drop_duplicates().copy()
    df['ticker'] = ticker
    df['Target_1Y'] = df['Adj Close'].shift(-100)/df['Adj Close'] - 1
    df_global = pd.concat([df_global, df[['Date', 'ticker', 'Adj Close', 'Target_1Y']]])
df_global.rename(columns = {'Date':'fecha_fin_cuatrimestre_fiscal'}, inplace = True)

0 501
[*********************100%***********************]  1 of 1 completed
1 501
[*********************100%***********************]  1 of 1 completed
2 501
[*********************100%***********************]  1 of 1 completed
3 501
[*********************100%***********************]  1 of 1 completed
4 501
[*********************100%***********************]  1 of 1 completed
5 501
[*********************100%***********************]  1 of 1 completed
6 501
[*********************100%***********************]  1 of 1 completed
7 501
[*********************100%***********************]  1 of 1 completed
8 501
[*********************100%***********************]  1 of 1 completed
9 501
[*********************100%***********************]  1 of 1 completed
10 501
[*********************100%***********************]  1 of 1 completed
11 501
[*********************100%***********************]  1 of 1 completed
12 501
[*********************100%***********************]  1 of 1 completed
13 501
[**************

[*********************100%***********************]  1 of 1 completed
212 501
[*********************100%***********************]  1 of 1 completed
213 501
[*********************100%***********************]  1 of 1 completed
214 501
[*********************100%***********************]  1 of 1 completed
215 501
[*********************100%***********************]  1 of 1 completed
216 501
[*********************100%***********************]  1 of 1 completed
217 501
[*********************100%***********************]  1 of 1 completed
218 501
[*********************100%***********************]  1 of 1 completed
219 501
[*********************100%***********************]  1 of 1 completed
220 501
[*********************100%***********************]  1 of 1 completed
221 501
[*********************100%***********************]  1 of 1 completed
222 501
[*********************100%***********************]  1 of 1 completed
223 501
[*********************100%***********************]  1 of 1 completed
224 501

[*********************100%***********************]  1 of 1 completed
424 501
[*********************100%***********************]  1 of 1 completed
425 501
[*********************100%***********************]  1 of 1 completed
426 501
[*********************100%***********************]  1 of 1 completed
427 501
[*********************100%***********************]  1 of 1 completed
428 501
[*********************100%***********************]  1 of 1 completed
429 501
[*********************100%***********************]  1 of 1 completed
430 501
[*********************100%***********************]  1 of 1 completed
431 501
[*********************100%***********************]  1 of 1 completed
432 501
[*********************100%***********************]  1 of 1 completed
433 501
[*********************100%***********************]  1 of 1 completed
434 501
[*********************100%***********************]  1 of 1 completed
435 501
[*********************100%***********************]  1 of 1 completed
436 501

In [5]:
# Se combinan para construir métricas

full_data_balances.rename(columns = {'bookValue':'Book Value, M$'}, inplace = True)
full_data_balances.rename(columns = {'pb':'Price to book, %'}, inplace = True)
full_data_balances.rename(columns = {'quickRatio':'Quick Ratio (Assets Inventory/Liability), %'}, inplace = True)
full_data_balances.rename(columns = {'currentRatio':'Current Ratio (Assets/Liability), %'}, inplace = True)
full_data_balances.rename(columns = {'roaTTM':'ROA (Returns/Assets), %'}, inplace = True)
full_data_balances.rename(columns = {'roeTTM':'ROE (Returns/Equity), %'}, inplace = True)
full_data_balances.rename(columns = {'roicTTM':'ROI (Returns/Investment), %'}, inplace = True)
full_data_balances.rename(columns = {'longtermDebtTotalEquity':'Debt-to-equity (Debt/Equity), %'}, inplace = True)
full_data_balances.rename(columns = {'longtermDebtTotalAsset':'Debt-to-asset (Debt/Assets), %'}, inplace = True)
full_data_balances.rename(columns = {'grossMargin':'Gross Margin (Profit/Revenue), %'}, inplace = True)
full_data_balances.rename(columns = {'operatingMargin':'Operating Margin (Income/Revenue), %'}, inplace = True)
full_data_balances.rename(columns = {'ebitPerShare':'EBIT per Share, $'}, inplace = True)
full_data_balances['Market Cap, B$'] = full_data_balances['Price to book, %'] * full_data_balances['Book Value, M$'] / 1000
full_data_balances.rename(columns = {'eps':'Earnings per share, $'}, inplace = True)
full_data_balances = pd.merge_asof(full_data_balances.sort_values(by='fecha_fin_cuatrimestre_fiscal'),df_global.sort_values(by='fecha_fin_cuatrimestre_fiscal'),on='fecha_fin_cuatrimestre_fiscal', by='ticker', direction='backward')
full_data_balances['Total Shares, M#'] = (full_data_balances['Market Cap, B$'] * 1000)/full_data_balances['Adj Close']
full_data_balances['Total earnings, M$'] = full_data_balances['Total Shares, M#'] * full_data_balances['Earnings per share, $']
full_data_balances['TTM Net Earnings per share, $'] = full_data_balances['Earnings per share, $'].rolling(window=4, axis='rows').sum()
full_data_balances['Price Earnings Ratio, $'] = full_data_balances['Adj Close']/full_data_balances['TTM Net Earnings per share, $']
full_data_balances['Earnings before interest and taxes (EBIT), M$'] = full_data_balances['EBIT per Share, $'] * full_data_balances['Total Shares, M#']

'''full_data_balances['Assets, M$'] = full_data_balances['Total earnings, M$'] / full_data_balances['ROA (Returns/Assets), %']
full_data_balances['Liability, M$'] = full_data_balances['Assets, M$'] / full_data_balances['Current Ratio (Assets/Liability), %']
full_data_balances['Assets Inventory, M$'] = full_data_balances['Liability, M$'] * full_data_balances['Quick Ratio (Assets Inventory/Liability), %']
full_data_balances['Equity, M$'] = full_data_balances['Total earnings, M$'] / full_data_balances['ROE (Returns/Equity), %']
full_data_balances['Investment, M$'] = full_data_balances['Total earnings, M$'] / full_data_balances['ROI (Returns/Investment), %']
full_data_balances['Long Term Debt 1, M$'] = full_data_balances['Equity, M$'] / full_data_balances['Debt-to-equity (Debt/Equity), %']
full_data_balances['Long Term Debt 2, M$'] = full_data_balances['Assets, M$'] / full_data_balances['Debt-to-asset (Debt/Assets), %']'''


"full_data_balances['Assets, M$'] = full_data_balances['Total earnings, M$'] / full_data_balances['ROA (Returns/Assets), %']\nfull_data_balances['Liability, M$'] = full_data_balances['Assets, M$'] / full_data_balances['Current Ratio (Assets/Liability), %']\nfull_data_balances['Assets Inventory, M$'] = full_data_balances['Liability, M$'] * full_data_balances['Quick Ratio (Assets Inventory/Liability), %']\nfull_data_balances['Equity, M$'] = full_data_balances['Total earnings, M$'] / full_data_balances['ROE (Returns/Equity), %']\nfull_data_balances['Investment, M$'] = full_data_balances['Total earnings, M$'] / full_data_balances['ROI (Returns/Investment), %']\nfull_data_balances['Long Term Debt 1, M$'] = full_data_balances['Equity, M$'] / full_data_balances['Debt-to-equity (Debt/Equity), %']\nfull_data_balances['Long Term Debt 2, M$'] = full_data_balances['Assets, M$'] / full_data_balances['Debt-to-asset (Debt/Assets), %']"

In [6]:
pd.options.display.max_columns = 100
full_data_balances = full_data_balances[['fecha_aproximada_presentacion',
 'ticker',
 'fecha_fin_cuatrimestre_fiscal',
 'Book Value, M$',
 'Current Ratio (Assets/Liability), %',
 'Earnings before interest and taxes (EBIT), M$',
 'EBIT per Share, $',
 'Earnings per share, $',
 'Gross Margin (Profit/Revenue), %',
 'Debt-to-asset (Debt/Assets), %',
 'Debt-to-equity (Debt/Equity), %',
 'Operating Margin (Income/Revenue), %',
 'Price to book, %',
 'Quick Ratio (Assets Inventory/Liability), %',
 'ROA (Returns/Assets), %',
 'ROE (Returns/Equity), %',
 'ROI (Returns/Investment), %',
 'Market Cap, B$',
 'Total Shares, M#',
 'Total earnings, M$',
 'TTM Net Earnings per share, $',
 'Price Earnings Ratio, $',
 'Target_1Y'
 ]].sort_values(by=['ticker', 'fecha_aproximada_presentacion'])

full_data_balances.to_csv('resumen_balances.csv', index=False)

In [2]:
full_data_balances = pd.read_csv('resumen_balances.csv')

# Análisis simple

In [4]:
lags = 8

analisis = pd.DataFrame(columns=['Columna', 
                                 'Lags', 
                                 'Mayor/Menor a la media de balance', 
                                 'Cantidad', 
                                 '% medio a 1 año'])

for col in full_data_balances.columns[3:-1]:
    
    string = col + '_var'
    full_data_balances[string] = np.where(full_data_balances['ticker'] != full_data_balances['ticker'].shift(lags-1), np.nan,
                                np.where(full_data_balances[col] > full_data_balances[col].rolling(window=lags, axis='rows').mean(),
                                         "Mayor a media", 
                                np.where(full_data_balances[col] < full_data_balances[col].rolling(window=lags, axis='rows').mean(),                                         
                                         "Menor a media",
                                         "NA")))
    
    analisis.loc[len(analisis)]=[col, 
                                 lags, 
                                 'Mayor', 
                                 len(full_data_balances[full_data_balances[string] == "Mayor a media"]), 
                                 round(full_data_balances[full_data_balances[string] == "Mayor a media"].Target_1Y.mean(), 5)]
    
    analisis.loc[len(analisis)]=[col, 
                                 lags, 
                                 'Menor', 
                                 len(full_data_balances[full_data_balances[string] == "Menor a media"]), 
                                 round(full_data_balances[full_data_balances[string] == "Menor a media"].Target_1Y.mean(), 5)]

In [5]:
analisis.sort_values(by='% medio a 1 año')

,Columna,Lags,Mayor/Menor a la media de balance,Cantidad,% medio a 1 año
18,"Price to book, %",8,Mayor,18787,0.05618
28,"Market Cap, B$",8,Mayor,24034,0.05618
24,"ROE (Returns/Equity), %",8,Mayor,12534,0.05942
33,"Total earnings, M$",8,Menor,12334,0.06015
36,"Price Earnings Ratio, $",8,Mayor,3787,0.06059
22,"ROA (Returns/Assets), %",8,Mayor,13025,0.06106
26,"ROI (Returns/Investment), %",8,Mayor,13021,0.06112
35,"TTM Net Earnings per share, $",8,Menor,3522,0.06122
31,"Total Shares, M#",8,Menor,24014,0.06194
5,"Earnings before interest and taxes (EBIT), M$",8,Menor,11772,0.06203


Verificación fuerte del análisis fundamental. Los signos son concordantes con lo que se espera, por ejemplo:
* Empresas con Gross Margin y Operating Margin alto suben más
* Empresas con Price to book bajo suben más
* Empresas con Quick Ratio y Current Ratio alto suben más
* Empresas con Earnings per share alto suben más
* Empresas con Debt-to-equity bajo suben más

Para no overfitear, entre esas reglas debemos llegar a alguna más simple que deje una buena parte de los casos dentro y que además genere una buena performance

In [16]:
pb_menor = full_data_balances['Price to book, %_var'] == 'Menor a media'
gm_mayor = full_data_balances['Gross Margin (Profit/Revenue), %_var'] == 'Mayor a media'
qr_mayor = full_data_balances['Quick Ratio (Assets Inventory/Liability), %_var'] == 'Mayor a media'

print('TOTAL DE OBSERVACIONES:')
print('Cantidad: ', len(full_data_balances))
print('Retorno medio a un año:', round(full_data_balances['Target_1Y'].mean(), 4))

print('OBSERVACIONES CON BUEN BALANCE:')
print('Cantidad: ', len(full_data_balances[(gm_mayor|qr_mayor)&(pb_menor)]))
print('Retorno medio a un año:', round(full_data_balances[(gm_mayor|qr_mayor)&(pb_menor)]['Target_1Y'].mean(), 4))

full_data_balances['Posible_compra'] = np.where((gm_mayor|qr_mayor)&(pb_menor),
                                               "Sí",
                                               "No")

full_data_balances['Date'] = full_data_balances['fecha_aproximada_presentacion']

TOTAL DE OBSERVACIONES:
Cantidad:  41119
Retorno medio a un año: 0.0714
OBSERVACIONES CON BUEN BALANCE:
Cantidad:  10054
Retorno medio a un año: 0.0888


In [18]:
full_data_balances[['ticker', 'Date', 'Posible_compra']].to_csv('analisis_fundamental.csv')